In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import timm
import numpy as np
import pandas as pd

from torchvision import transforms
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# paths
ZIP_PATH = "/content/drive/MyDrive/data_extracted/faces.zip"
EXTRACT_TO = "/content/drive/MyDrive/data_extracted"

# create clean target folder
# !mkdir -p "$EXTRACT_TO"

# extract
# !unzip -q "$ZIP_PATH" -d "$EXTRACT_TO"

print("Extraction done.")


Using device: cuda
Extraction done.


In [11]:
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # faces often grayscale
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [12]:
DATA_DIR = "/kaggle/input/fer2013"
TRAIN_DIR = DATA_DIR + "/train"
TEST_DIR  = DATA_DIR + "/test"

# class names (folder names = labels)
classes = sorted(os.listdir(TRAIN_DIR))
class_to_idx = {cls: i for i, cls in enumerate(classes)}
print("Classes:", classes)


# Create datasets automatically from folder structure
train_dataset = ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_dataset  = ImageFolder(root=TEST_DIR,  transform=test_transform)

# Create loaders
# shuffle=True is what fixes your low accuracy!
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

print(f"Total batches: {len(train_loader)}")

Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Total batches: 898


In [13]:
# MANUAL BATCH LOADER
def get_batch(files, labels, transform, batch_size, start):
    imgs, labs = [], []
    end = min(start + batch_size, len(files))

    for i in range(start, end):
        imgs.append(load_image(files[i], transform))
        labs.append(torch.tensor(labels[i]))

    return (
        torch.stack(imgs).to(device, non_blocking=True),
        torch.stack(labs).to(device, non_blocking=True)
    )

# MODEL (ResNet18)
num_classes = len(classes)

model = timm.create_model("resnet18", pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [14]:
# TRAIN & EVAL FUNCTIONS

def train_epoch(loader, model, optimizer, criterion):
    model.train()
    correct = total = 0
    
    for imgs, labs in loader:
        imgs, labs = imgs.to(device), labs.to(device)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labs)
        loss.backward()
        optimizer.step()

        preds = out.argmax(1)
        correct += (preds == labs).sum().item()
        total += labs.size(0)

    return correct / total

def evaluate(loader, model):
    model.eval()
    preds_all, labels_all = [], []

    with torch.no_grad():
        for imgs, labs in loader:
            imgs = imgs.to(device)
            out = model(imgs)
            
            preds_all.append(out.argmax(1).cpu())
            labels_all.append(labs.cpu())

    return (
        torch.cat(labels_all).numpy(),
        torch.cat(preds_all).numpy()
    )


# PARTIAL FINE-TUNING (classifier only)

for p in model.parameters():
    p.requires_grad = False
for p in model.fc.parameters():
    p.requires_grad = True

optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

print("\nPartial Fine-Tuning")
for epoch in range(3):
    acc = train_epoch(train_loader, model, optimizer, criterion)
    print(f"Epoch {epoch+1}: Train Acc = {acc:.4f}")



Partial Fine-Tuning
Epoch 1: Train Acc = 0.2808
Epoch 2: Train Acc = 0.3121
Epoch 3: Train Acc = 0.3233


In [15]:


for p in model.parameters():
    p.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=5e-5)

print("\nFull Fine-Tuning")
for epoch in range(5):
    acc = train_epoch(train_loader, model, optimizer, criterion)
    print(f"Epoch {epoch+1}: Train Acc = {acc:.4f}")

# FINAL TEST METRICS

y_true, y_pred = evaluate(test_loader, model)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=classes))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Full Fine-Tuning
Epoch 1: Train Acc = 0.3601
Epoch 2: Train Acc = 0.3983
Epoch 3: Train Acc = 0.4289
Epoch 4: Train Acc = 0.4482
Epoch 5: Train Acc = 0.4677

Classification Report:
              precision    recall  f1-score   support

       angry       0.39      0.48      0.43       958
     disgust       0.00      0.00      0.00       111
        fear       0.31      0.13      0.18      1024
       happy       0.76      0.83      0.79      1774
     neutral       0.48      0.55      0.52      1233
         sad       0.44      0.44      0.44      1247
    surprise       0.63      0.74      0.68       831

    accuracy                           0.55      7178
   macro avg       0.43      0.45      0.43      7178
weighted avg       0.52      0.55      0.52      7178

Confusion Matrix:
[[ 462    0   66   76  149  166   39]
 [  66    0    3   13    3   24    2]
 [ 223    0  130   81  174  230  186]
 [  83    0   20 1479   87   43   62]
 [ 101    0   47  134  683  221   47]
 [ 201    0  

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
